In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
census = pd.read_csv('data/census_data.csv')

In [ ]:
census.head()

In [ ]:
## Transform categorical data into numerical

cat_columns = census.select_dtypes(['object']).columns
census[cat_columns] = census[cat_columns].astype('category')
census[cat_columns] = census[cat_columns].apply(lambda x: x.cat.codes)

In [ ]:
census.head()

## Train and test split

In [ ]:
# Train and test split
x_data = census.drop('income_bracket',axis=1)
labels = census['income_bracket']

x_train, x_test, y_train, y_test = train_test_split(x_data,labels,test_size = 0.3)

In [ ]:
# Create fetuare columns
feat_cols = []
for column in x_data.columns:
    feat_cols.append(tf.feature_column.numeric_column(column))
    #if(str(census[column].dtype) == "int64"):   
    #else:
        #feat_cols.append(tf.feature_column.categorical_column_with_hash_bucket(column,hash_bucket_size=10))

In [ ]:

input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train,batch_size=100,num_epochs=None,shuffle=True)
dnn_model = tf.estimator.DNNClassifier(hidden_units=[13,15,13],feature_columns=feat_cols,n_classes=2)
dnn_model.train(input_fn=input_func,steps=10000)

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test, y=y_test,batch_size=100,num_epochs=1,shuffle=False)
dnn_model.evaluate(eval_input_func)

In [ ]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test),num_epochs=1,shuffle=False)
pred_gen = dnn_model.predict(pred_input_func)
predictions = list(pred_gen)

final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [ ]:
 print(classification_report(y_test,final_preds))